<a href="https://colab.research.google.com/github/mariivalverde/An-lisis-de-datos/blob/main/CIENCIA_DE_DATOS_10_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
!pip install dash pandas plotly


import pandas as pd
import dash
from dash import html, dcc
import plotly.express as px
from dash.dependencies import Input, Output

# Cargar datos
spacex_df = pd.read_csv("spacex_launch_dash.csv")

# Crear app
app = dash.Dash(__name__)

# Lista de sitios únicos
launch_sites = spacex_df['Launch Site'].unique()
site_options = [{'label': site, 'value': site} for site in launch_sites]
site_options.insert(0, {'label': 'Todos los sitios', 'value': 'ALL'})

# Layout
app.layout = html.Div(children=[
    html.H1('SpaceX Launch Records Dashboard', style={'textAlign': 'center'}),

    # Dropdown para seleccionar sitio
    dcc.Dropdown(id='site-dropdown',
                 options=site_options,
                 value='ALL',
                 placeholder='Seleccione un sitio de lanzamiento',
                 searchable=True),

    # Gráfico de pastel
    html.Div(dcc.Graph(id='success-pie-chart')),

    # Slider de rango de payload
    html.P("Seleccione el rango de carga útil (kg):"),
    dcc.RangeSlider(id='payload-slider',
                    min=spacex_df['Payload Mass (kg)'].min(),
                    max=spacex_df['Payload Mass (kg)'].max(),
                    step=100,
                    value=[spacex_df['Payload Mass (kg)'].min(), spacex_df['Payload Mass (kg)'].max()],
                    marks={int(i): str(int(i)) for i in range(0, int(spacex_df['Payload Mass (kg)'].max())+1, 2000)}),

    # Gráfico de dispersión
    html.Div(dcc.Graph(id='success-payload-scatter')),
])

# Callbacks

# Pie chart
@app.callback(Output('success-pie-chart', 'figure'),
              Input('site-dropdown', 'value'))
def update_pie_chart(site):
    if site == 'ALL':
        fig = px.pie(spacex_df, names='Launch Site', title='Total de Éxitos por Sitio de Lanzamiento')
    else:
        filtered_df = spacex_df[spacex_df['Launch Site'] == site]
        fig = px.pie(filtered_df, names='class', title=f'Éxito vs Falla en {site}',
                     color_discrete_map={1: 'green', 0: 'red'},
                     labels={'class': 'Resultado'})
    return fig

# Scatter Plot
@app.callback(Output('success-payload-scatter', 'figure'),
              [Input('site-dropdown', 'value'),
               Input('payload-slider', 'value')])
def update_scatter(site, payload_range):
    low, high = payload_range
    filtered_df = spacex_df[(spacex_df['Payload Mass (kg)'] >= low) &
                            (spacex_df['Payload Mass (kg)'] <= high)]

    if site != 'ALL':
        filtered_df = filtered_df[filtered_df['Launch Site'] == site]

    fig = px.scatter(filtered_df, x='Payload Mass (kg)', y='class',
                     color='Booster Version',
                     title='Payload vs Resultado del Lanzamiento',
                     labels={'class': 'Resultado'})
    return fig

# Ejecutar app
if __name__ == '__main__':
    app.run(debug=True)


<IPython.core.display.Javascript object>